In [28]:
import urllib.request
import json
import os
import ssl
import pandas as pd

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context



def get_data(df,i):
    features = ['age','hypertension','heart_disease','bmi','HbA1c_level','blood_glucose_level']
    input_data = df[features].iloc[201].to_list()

    data =  {
  "input_data": [input_data],
  "params": {}
    }

    body = str.encode(json.dumps(data))
    return body



def call_endpoint(deployment_name,url,api_key,body):

    if not api_key:
        raise Exception("A key should be provided to invoke the endpoint")

    headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key), 'azureml-model-deployment': deployment_name }
    req = urllib.request.Request(url, body, headers)

    try:
        response = urllib.request.urlopen(req)

        result = response.read()
        print(result)
    except urllib.error.HTTPError as error:
        print("The request failed with status code: " + str(error.code))
    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
        print(error.info())
        print(error.read().decode("utf8", 'ignore'))

In [29]:

from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient.from_config(credential=DefaultAzureCredential())
data_asset = ml_client.data.get("diabates_pre_processed", version="1")

df = pd.read_csv(data_asset.path)

Found the config file in: /config.json


In [30]:
# this line is needed if you use self-signed certificate in your scoring service.
allowSelfSignedHttps(True) 

#generate the json input to your endpoint
body = get_data(df,309)

#call the api
deployment_name = 'diabetes-lr-1' #your deployment name here
url = 'https://mlstudio-test-1-rsmiw.australiaeast.inference.ml.azure.com/score' #your endpoint url here
api_key = 'jzlwWm55Sq36d7wWZUetNH8j8Ei1v6Ff' #primary or secondary API key here 

call_endpoint(deployment_name,url,api_key,body)

b'[0]'
